In [24]:
from eventregistry import *
import numpy as np
import pandas as pd

# Print a list of recently added articles mentioning entered words
api_key = 'eda39267-9017-481a-860d-0b565c6d8bf3'
er = EventRegistry(apiKey = api_key)

def get_articles(keywords): 
    q = QueryArticlesIter(keywords=QueryItems.AND(keywords))
    
    x = 0
    df = pd.DataFrame({'source':'test','url':'testing','text':'placeholers'}, index=[0])
    df.columns = ['source','url','text']
    
    for article in q.execQuery(er, sortBy = 'date'):
        data = {
            'source': article['source']['title'].encode('utf-8'),
#             'title' : article['title'].encode('utf-8'),
            'url' : article['url'].encode('utf-8'),
            'text' : article['body'].encode('utf-8')
        }
        df_temp = pd.DataFrame(data,index=[x])
        df = pd.concat([df,df_temp])
        x += 1
    return df

df = pd.DataFrame({'source':'test','url':'testing','text':'placeholers'}, index=[0])
df.columns = ['source','url','text']
params = [['embassy', 'zone', 'decade'],['digging', 'expands', 'decade']]

for query in params:
    df = pd.concat([df,get_articles(query)])

df = df.drop(df.index[[0,1]])
df = df.reset_index(drop=True)
df.to_json(orient='index')

using user provided API key for making requests
Event Registry host: http://eventregistry.org
['embassy', 'zone', 'decade']
['digging', 'expands', 'decade']


In [180]:
df.to_csv('articles.csv')


In [37]:
df = df.reset_index(drop=True)


In [38]:
df.to_json(orient='index')

'{"0":{"source":"The Boston Globe","text":"KABUL -- A huge public works project over the next two years will reshape the center of the Afghan capital to bring nearly all Western embassies, major government ministries, and NATO and US military headquarters within the Green Zone security district\\n\\nAfter 16 years of American presence in Kabul, it is a stark acknowledgment that even the city\'s central districts have become too difficult to defend from Taliban bombings.\\n\\nAdvertisement\\n\\nSoon, US Embassy employees in Kabul will no longer need to take a Chinook helicopter to cross the street to a military base less than 100 yards outside the present protection zone.\\n\\nInstead, the boundaries of the Green Zone will be redrawn to include that base, known as the Kabul City Compound, formerly the headquarters for US Special Operations forces in the capital. The zone is separated from the rest of the city of 5 million by a network of police, military, and private security checkpoint

In [27]:
df = df.drop(df.index[[0,1]])
df

,source,text,url
1,The Boston Globe,KABUL -- A huge public works project over the ...,https://www.bostonglobe.com/news/world/2017/09...
2,The New York Times,But the capital project is also clearly taking...,https://www.nytimes.com/2017/09/16/world/asia/...
3,Asia Times,The emerging humanitarian crisis that has been...,http://www.atimes.com/article/turkey-rohingyas...
4,southasiajournal.net,Turkish Muslims pray near Fatih Mosque in Ista...,http://southasiajournal.net/turkey-the-rohingy...
5,To Inform is to Influence,"Anonymous expert compilation, analysis, and re...",https://toinformistoinfluence.com/2017/09/15/r...
6,The Independent,Tens of thousands of coastal residents evacuat...,http://www.independent.co.uk/news/world/asia/t...
7,Eurasia Review,"A new report by a retired IT executive at IBM,...",http://www.eurasiareview.com/15092017-the-russ...
8,asiancorrespondent.com,THE EMERGING humanitarian crisis that has been...,https://asiancorrespondent.com/2017/09/rohingy...
9,The Wire,Children recycle goods from the ruins of a mar...,https://thewire.in/177674/rohingya-crisis-turk...
10,iran-daily.com,Afghanistan's Foreign Minister Salahuddin Rabb...,http://www.iran-daily.com/News/200364.html


In [ ]:
df.shape

In [8]:
df1 = pd.read_csv('articles.csv')
df1.columns = ['id','source','text',"""'title""" 'url']

bodies = df1.loc[:,['id','text']]
bodies.to_csv('bodies.csv')

# we want claim here, not title
headlines = df1.loc[:,['id','title']]
headlines.to_csv('headlines.csv')

urls = df1.loc[:,['id','source','url']]
urls.to_csv('url.csv')

In [9]:
bodies

,id,text
0,0,Sometimes it's time to move on. And while some...
1,1,"It's not always an easy decision, and in most ..."
2,2,Olmsted Medical Center is in the midst of a le...
3,3,Artificial intelligence - commonly known as A....
4,4,Duke University researchers have developed a h...
5,5,During the Sunday school class in a Pentecosta...
6,6,A polyamory evangelist with a genius IQ said h...
7,7,The power holders have almost unrestricted con...
8,8,Honolulu: After eight months of living in isol...
9,9,Honolulu: After eight months of living in isol...


In [156]:
from newspaper import Article

def get_keywords(user_url):
    url = user_url.decode('utf-8')
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()

    keywords = article.keywords
    kl = []
    for word in keywords:
        kl.append(word.encode('utf-8'))
    return kl
url = 'https://www.nytimes.com/2017/09/16/world/asia/kabul-green-zone-afghanistan.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news&mtrref=www.nytimes.com&gwh=044CBF2516278CC5DE5B7C307F5B96F1&gwt=pay'
kl = get_keywords(url)
len(kl)


13

In [157]:
import random

def get_search_params(keywords):
    search_params = []
    while len(keywords) != 0:
        # Randomly select 3 words
        rm = random.sample(keywords,3)
        # add the list of 3 words to the searchable list
        search_params.append(rm)
        # remove words from the list
        for word in rm:
            keywords.remove(word)

        # put 1 or 2 random words back
        # if 3 words left just append to search_params
        if len(keywords) is 3:
            search_params.append(keywords)
            keywords = []
        # if no words left just exit
        elif len(keywords) is 0:
            keywords = []
        # if 1 word left, append 2 and search_params
        elif len(keywords) is 1:
            keywords.append(random.sample(rm,2)[0:2])
        else:
            keywords.append(random.sample(rm,1)[0])
    return search_params

params = get_search_params(kl)

[['embassy', 'zone', 'decade'],
 ['american', 'mr', 'kabul'],
 ['afghan', 'trump', 'digging'],
 ['decade', 'afghanistan', 'green'],
 ['decade', 'security', 'mr'],
 ['digging', 'expands', 'decade']]

In [14]:
df = pd.DataFrame({'source':'test','url':'testing','text':'placeholers'},index=[0])
df.columns = ['source','url','text']

def helper(text):
    df1 = pd.DataFrame({'source':text,'url':text,'text':text},index=[1])
    return df1
    
df2 = helper('asdf')
# helper('jkl;')
df = pd.concat([df,df2])
df

,source,text,url
0,test,testing,placeholers
1,asdf,asdf,asdf


In [23]:
params = [['embassy', 'zone', 'decade'],['digging', 'expands', 'decade']]
params

[['embassy', 'zone', 'decade'], ['digging', 'expands', 'decade']]